In [1]:
from simple.binance import getHistMonth
from simple.plotly import interactFigure
from simple.pretty import pmap
import pandas as pd
from calendar import day_name

In [2]:
frame = '1h'
tickers = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'XRPUSDT']
weekdays = [f'{day: >20}' for day in day_name]

In [3]:
def getDiff(ticker):
    return getHistMonth('2019-01', '2021-09', ticker, frame, close_only=True).diff().dropna()

In [4]:
F = pd.concat(pmap(getDiff, tickers), axis=1)
F = F.join(F.index.isocalendar())
F['week_hour'] = F.groupby(['year', 'week']).cumcount()
F

  0%|          | 0/6 [00:00<?, ?it/s]

,BTCUSDT,ETHUSDT,LTCUSDT,TRXUSDT,ETCUSDT,XRPUSDT,year,week,day,week_hour
DT,,,,,,,,,,
2019-01-01 01:00:00,-10.62,-0.68,-0.22,-0.00013,-0.0261,-0.00109,2019,1,2,0
2019-01-01 02:00:00,0.31,-0.42,-0.02,0.00002,-0.0243,-0.00168,2019,1,2,1
2019-01-01 03:00:00,3.13,0.60,0.06,0.00014,0.0465,0.00234,2019,1,2,2
2019-01-01 04:00:00,-0.42,1.03,0.14,0.00013,0.0126,0.00179,2019,1,2,3
2019-01-01 05:00:00,7.23,0.40,0.09,0.00011,0.0528,0.00081,2019,1,2,4
...,...,...,...,...,...,...,...,...,...,...
2021-09-30 19:00:00,-461.77,-44.50,-2.00,-0.00072,-0.5700,-0.00980,2021,39,4,89
2021-09-30 20:00:00,-97.23,-19.41,-0.30,-0.00044,-0.1400,-0.00380,2021,39,4,90
2021-09-30 21:00:00,318.92,32.08,1.30,0.00057,0.4200,0.00860,2021,39,4,91


In [6]:
Hours = list(range(0, 168, 24))
Hours

[0, 24, 48, 72, 96, 120, 144]

In [7]:
def model(index: int = (0, len(tickers))):
    ticker = tickers[index]
    Profile = F.groupby('week_hour')[ticker].mean().cumsum()
    layout = dict(title={'text': ticker})
    return 0, locals()

In [8]:
box = interactFigure(model, {
    'Profile': dict(width=6, opacity=0.3),
    'Weekdays': dict(x=Hours, y=[0] * len(Hours), mode="text", textposition="top right", text=weekdays)
}, height=500)

In [9]:
# add vertical daily lines
[box.children[1].add_vline(x=hour, line_color='gray', line_dash='dot') for hour in Hours + [168]];

In [10]:
box

    '…